In [17]:
%run converter.ipynb

In [18]:
import requests
import re
from lxml import html
import urllib
from pathlib import Path
from bs4 import BeautifulSoup
from docx.api import Document

In [21]:
def get_annotation(doc_name: str) -> str:
    """Parse paragraphs in docx. Return annotation.
    """
    doc = Document(os.path.abspath('./articles/' + doc_name + '.docx'))
    tmp = ''
    annotation = ''
    ann_flag = 0
    for para in doc.paragraphs:
        if tmp.strip() == 'Аннотация' and not ann_flag:
            ann_flag = 1
        if ann_flag:
            if para.text.find('Ключевые слова') != -1:
                break
            annotation += para.text
        tmp = para.text
    return annotation

def download(url: str, annotation: str):
    """Download article and annotation from 'url' to directory 'articles'. 
    If folder doesn't exist, create it. Name file by its number.    
    """
    download.name += 1
    url = urllib.parse.quote(url)
    Path("./articles").mkdir(parents=True, exist_ok=True)
    urllib.request.urlretrieve(
        'http://www.rfej.ru' + url, './articles/{}.pdf'.format(download.name))
    to_del = conv.pdf_to_docx(str(download.name))  # convert to .docx
    os.remove(to_del)  # delete .pdf
    pdf_ann = get_annotation(str(download.name))
    if pdf_ann:  # if annotation is inside .pdf
        annotation = pdf_ann
    f = open('./articles/{}.txt'.format(download.name), "w", encoding='utf-8') 
    f.write(annotation) 
    f.close()
download.name = 0  # var for file names

def parse_articles(page_id: str):
    page = requests.get('http://www.rfej.ru{}'.format(page_id))
    webpage = html.fromstring(page.content)
    soup = BeautifulSoup(page.content, 'html.parser')
    a = soup.find_all('p', attrs={'class': 'sanno'})
    pdf_url = list(set(webpage.xpath("//a[contains(@href, '.pdf')]/@href")))
    if pdf_url and a[0].text:
        download(pdf_url[0], a[0].text)

def parse_months(month: str):
    page = requests.get('http://www.rfej.ru{}'.format(month))
    webpage = html.fromstring(page.content)
    for el in set(webpage.xpath("//a[starts-with(@href, '/rvv/id')]/@href")):
        parse_articles(el)

def parse_years(year: str):
    page = requests.get('http://www.rfej.ru{}'.format(year))
    webpage = html.fromstring(page.content)
    for el in set(webpage.xpath("//a[starts-with(@href, '/rvv/years/month')]/@href")):
        parse_months(el)

def parse():
    """Function parses in cycle every year/month/article page and downloads
    all articles and annotations from this pages. 
    """
    page = requests.get('http://www.rfej.ru/rvv/years')
    webpage = html.fromstring(page.content)
    for el in webpage.xpath(
        "//a[starts-with(@href, '/rvv/years/by') and text()>='2012']/@href"):
        parse_years(el)
        

In [ ]:
conv = Converter()
parse() # start parsing and downloading
del conv